La librería que vamos a usar para la automatización del monitoreo de los datos, es Watchdog. Estuvimos averiguando y es una muy buena herramienta para identificar cambios en el sistema de archivos en tiempo real. Es particularmente útil cuando necesitas realizar acciones automáticas en respuesta a eventos como la creación, modificación, o eliminación de archivos y directorios. Es por eso, que nos decidimos por Watchdog

Para interactual con la base de datos, vamos a usar pyodbc. Ver con cual resulta mas fácil

In [ ]:
# Conectarse a la base de datos en cuestion

import pyodbc
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la cadena de conexión
server = 'your_server_name.database.windows.net'
database = 'your_db_name'
username = 'your_username'
password = 'your_password'
driver = '{ODBC Driver 17 for SQL Server}'

# Conexión con pyodbc
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

# O, si prefieres usar SQLAlchemy para manejar la conexión y ejecutar el DataFrame
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')


In [ ]:
# Leer el último registro procesado

last_ingestion = pd.read_sql('SELECT TOP 1 * FROM ingestion_control ORDER BY last_ingestion_id DESC', conn)
last_id = last_ingestion['last_ingestion_id'].iloc[0] if not last_ingestion.empty else 0


In [ ]:
# Cargar el CSV en un DataFrame de pandas
df = pd.read_csv('path/to/your/data.csv')

In [ ]:

# Filtrar los nuevos registros que no han sido procesados
new_data = df[df['id_column'] > last_id]

In [ ]:
# Insertar los nuevos datos en la base de datos
if not new_data.empty:
    new_data.to_sql('your_table_name', engine, if_exists='append', index=False)

    # Actualizar la tabla de control con el último ID procesado
    last_processed_id = new_data['id_column'].max()
    engine.execute(f"INSERT INTO ingestion_control (last_ingestion_id, last_ingestion_timestamp) VALUES ({last_processed_id}, CURRENT_TIMESTAMP)")
else:
    print("No hay nuevos datos para insertar.")

## Alternativa para la ingesta de datos con GOOGLE CLOUD SERVICE

--------------------------------------------------------------------------

In [ ]:
import os
import csv
import pyodbc
from google.cloud import storage

def process_csv(event, context):
    # Información del archivo en GCS
    bucket_name = event['bucket']
    file_name = event['name']

    # Conectar a Google Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Descargar el archivo CSV a un archivo temporal
    local_file = f"/tmp/{file_name}"
    blob.download_to_filename(local_file)

    # Conectar a SQL Server
    server = 'tcp:mi-servidor-sql-server.database.windows.net'  # Cambiar por la dirección de tu servidor SQL Server
    database = 'mi_base_de_datos'  # Cambiar por el nombre de tu base de datos en SQL Server
    username = 'mi_usuario'  # Cambiar por el nombre de usuario de tu base de datos SQL Server
    password = 'mi_contraseña'  # Cambiar por la contraseña del usuario de tu base de datos SQL Server
    driver = '{ODBC Driver 17 for SQL Server}'  # Verificar que el driver ODBC esté disponible en el entorno

    conn = pyodbc.connect(
        'DRIVER=' + driver + ';SERVER=' + server + 
        ';PORT=1433;DATABASE=' + database + 
        ';UID=' + username + 
        ';PWD=' + password
    )
    cursor = conn.cursor()

    # Lógica de procesamiento dependiendo del archivo   
    if "ventas_limpio" in file_name:
        process_csv_with_executemany(local_file, cursor, 'ventas_tabla')  # Cambiar 'ventas_tabla' por el nombre de la tabla de ventas en SQL Server
    elif "compras_limpio" in file_name:
        process_csv_with_executemany(local_file, cursor, 'compras_tabla')  # Cambiar 'compras_tabla' por el nombre de la tabla de compras en SQL Server
    elif "inventario_inicial_limpio" in file_name:
        process_csv_with_executemany(local_file, cursor, 'inventario_inicial_tabla')  # Cambiar 'inventario_inicial_tabla' por el nombre de la tabla de inventario inicial en SQL Server
    elif "inventario_final_limpio" in file_name:
        process_csv_with_executemany(local_file, cursor, 'inventario_final_tabla')  # Cambiar 'inventario_final_tabla' por el nombre de la tabla de inventario final en SQL Server

    # Confirmar y cerrar la conexión
    conn.commit()
    cursor.close()
    conn.close()

    # Eliminar archivo temporal
    os.remove(local_file)

def process_csv_with_executemany(local_file, cursor, table_name):
    """
    Procesa un archivo CSV e inserta los datos en la tabla SQL Server utilizando executemany.
    :param local_file: Ruta del archivo CSV local.
    :param cursor: Cursor de la base de datos SQL Server.
    :param table_name: Nombre de la tabla en SQL Server.
    """
    with open(local_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Leer los encabezados
        data = [tuple(row) for row in reader]  # Capturar todas las columnas como tuplas

    # Construir la consulta SQL
    placeholders = ', '.join(['?' for _ in headers])
    query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    
    # Ejecutar la inserción masiva
    cursor.executemany(query, data)


#### Esto es para poner en la terminal cuando tenga acceso a CGS

gcloud functions deploy process_csv \
    --runtime python310 \
    --trigger-resource tu-bucket \
    --trigger-event google.storage.object.finalize \
    --timeout=540s \
    --entry-point process_csv


--runtime python310: Especifica la versión de Python.

--trigger-resource: El nombre del bucket en GCS donde se suben los archivos.

--trigger-event: Especifica que la función se dispare cuando un archivo se sube completamente.

--timeout=540s: Ajusta el tiempo máximo que la función puede ejecutarse.

--entry-point process_csv: Define la función principal que será ejecutada.

--------------------------------------------------------------------------

¿Qué Hacer Después?

Obtener Credenciales: Cuando tengas acceso a las credenciales de SQL Server y GCS, podrás completar las partes del código que dependen de estas.

Pruebas Locales: Puedes configurar un entorno local que imite el comportamiento de la función en la nube para hacer pruebas preliminares.

Despliegue y Monitoreo: Cuando todo esté listo, despliega la función en Google Cloud y monitorea su funcionamiento usando Cloud Logging y Monitoring.

------------

## En caso de querer probar primero GCS y despues integrar lo de SQL

Este codigo es una opcion donde no esta la parte de SQL integrada, asi probamos la funcionalidad de GCS

Primero la parte CGS

In [ ]:
import os
import csv
from google.cloud import storage

def process_csv(event, context):
    # Información del archivo en GCS
    bucket_name = event['bucket']
    file_name = event['name']

    # Conectar a Google Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Descargar el archivo CSV a un archivo temporal
    local_file = f"/tmp/{file_name}"
    blob.download_to_filename(local_file)

    # Lógica de procesamiento dependiendo del archivo
    if "ventas_limpio" in file_name:
        data = read_csv_file(local_file)
        # Aquí llamarías a la función SQL cuando esté integrada, por ahora solo imprime
        print("Datos procesados para ventas:", data)
    elif "compras_limpio" in file_name:
        data = read_csv_file(local_file)
        print("Datos procesados para compras:", data)
    elif "inventario_inicial_limpio" in file_name:
        data = read_csv_file(local_file)
        print("Datos procesados para inventario inicial:", data)
    elif "inventario_final_limpio" in file_name:
        data = read_csv_file(local_file)
        print("Datos procesados para inventario final:", data)

    # Eliminar archivo temporal
    os.remove(local_file)

def read_csv_file(local_file):
    """
    Lee el archivo CSV y retorna los datos como una lista de tuplas.
    :param local_file: Ruta del archivo CSV local.
    :return: Lista de tuplas con los datos.
    """
    with open(local_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Leer los encabezados
        data = [tuple(row) for row in reader]  # Capturar todas las columnas como tuplas
    return data


Integracion del SQL

In [ ]:
import pyodbc

def insert_data_into_sql(data, table_name):
    """
    Inserta datos en la tabla SQL Server especificada.
    :param data: Lista de tuplas con los datos a insertar.
    :param table_name: Nombre de la tabla en SQL Server.
    """
    # Conectar a SQL Server
    server = 'tcp:mi-servidor-sql-server.database.windows.net'  # Cambiar por la dirección de tu servidor SQL Server
    database = 'mi_base_de_datos'  # Cambiar por el nombre de tu base de datos en SQL Server
    username = 'mi_usuario'  # Cambiar por el nombre de usuario de tu base de datos SQL Server
    password = 'mi_contraseña'  # Cambiar por la contraseña del usuario de tu base de datos SQL Server
    driver = '{ODBC Driver 17 for SQL Server}'  # Verificar que el driver ODBC esté disponible en el entorno

    conn = pyodbc.connect(
        'DRIVER=' + driver + ';SERVER=' + server + 
        ';PORT=1433;DATABASE=' + database + 
        ';UID=' + username + 
        ';PWD=' + password
    )
    cursor = conn.cursor()

    # Construir la consulta SQL
    placeholders = ', '.join(['?' for _ in data[0]])
    query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    
    # Ejecutar la inserción masiva
    cursor.executemany(query, data)

    # Confirmar y cerrar la conexión
    conn.commit()
    cursor.close()
    conn.close()
